<a href="https://colab.research.google.com/github/YuzhouLin/current_proj/blob/newbranch/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/My\ Drive/

/content/drive/My Drive


In [3]:
!git clone https://github.com/YuzhouLin/current_proj.git

Cloning into 'current_proj'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 118 (delta 33), reused 114 (delta 29), pack-reused 0
Receiving objects: 100% (118/118), 4.35 MiB | 12.66 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
import numpy as np
import scipy.special as sc
import torch

In [ ]:
a = {
    'a':1,
    'b':2,
    'c':3,
    'd':4
}

b= ["c","d"]

#out = {item:a.get(item) for item in b}

out = {'b':a.get('b')}

out.update({item:a.get(item) for item in b})

#out[item] = a.get(item) for item in b
print(out)

{'b': 2, 'c': 3, 'd': 4}


In [ ]:
###  Calculate Vacuity and dissonance 

def Bal(a,b):
    if a*b !=0:
        output = 1-abs(a-b)/(a+b)
    else:
        output = 0
    return output        

#evidence = np.array([99]*12)
evidence = np.array([4,9,0])
alpha = evidence + 1
prob_SL = alpha / np.sum(alpha)
class_num = len(evidence)
total_evidence = np.sum(alpha)

u_va = class_num / total_evidence

print('The vacuity is: %.2f' %(u_va))

belief = evidence/total_evidence
belief = belief.tolist()
u_dis = 0.0
for index_k,k in enumerate(belief):
    temp0 = 0.0
    temp1 = 0.0
    for index_j,j in enumerate(belief):
        if index_j!=index_k:
            temp0 += j*Bal(k,j)
            temp1 += j
    if temp1!=0:
        u_dis += k*temp0/temp1
print('The dissonance is: %.2f' %(u_dis))

The vacuity is: 0.19
The dissonance is: 0.50


In [ ]:
import torch
#alpha = torch.ones([1, 12], dtype=torch.float32)
belief = torch.tensor([[0.3,0.3,0.0],[0.1,0.1,0.1]], dtype=torch.float32,requires_grad=True)
k_class = 3

'''
def Bal(a,b,batch_n):
    output = torch.where(a*b>0,1.0-torch.abs(a-b)/(a+b),0)
    if a*b !=0:
        output = 1.0-torch.abs(a-b)/(a+b)
    else:
        output = torch.Tensor.zeros(batch_n,1)
    return output
'''

batch_n = belief.size()[0]
u_dis = torch.zeros(batch_n,1)
for index_k in range(k_class):
    temp0 = torch.zeros(batch_n,1)
    temp1 = torch.zeros(batch_n,1)
    for index_j in range(k_class):
        if index_j!=index_k:
            k = belief[:,index_k].reshape(batch_n ,1)
            j = belief[:,index_j].reshape(batch_n ,1)
            temp0 += j*(1.0-torch.abs(k-j)/(k+j+1e-8))
            #torch.where(k*j>0.0,1.0-torch.abs(k-j)/(k+j),torch.zeros(batch_n,1))
            temp1 += j
    u_dis += k*temp0/(temp1+1e-8)

u_dis.sum().backward()
print(belief.grad.data)

tensor([[ 1.0000,  1.0000, -2.0000],
        [ 1.0000,  1.0000,  1.0000]])


In [ ]:
import numpy as np
a = np.array([[1.0],[2.0],[3.0]])
b = np.array([[2.0],[1.0],[10.0]])
c = np.concatenate((a,b),axis=1)
print(np.shape(a))
d = np.max(c,axis=1,keepdims=True)
print(np.shape(d))
print(d)

(3, 1)
(3, 1)
[[ 2.]
 [ 2.]
 [10.]]


In [ ]:
import torch
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]
 
w = torch.Tensor([1.0]) # w的初值为1.0
w.requires_grad = True # 需要计算梯度


#print(w.grad.data.type())



def forward(x):
    return x*w  # w是一个Tensor
 
 
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y)**2
 
print("predict (before training)", 4, forward(4).item())
 
for epoch in range(10):
    for x, y in zip(x_data, y_data):
        l =loss(x,y) # l是一个张量，tensor主要是在建立计算图 forward, compute the loss
        l.backward() #  backward,compute grad for Tensor whose requires_grad set to True

        print(w.data.type())
        print(w.grad.data.type())
        break
        print('\tgrad:', x, y, w.grad.item())
        w.data = w.data - 0.01 * w.grad.data   # 权重更新时，需要用到标量，注意grad也是一个tensor
 
        w.grad.data.zero_() # after update, remember set the grad to zero
 
    print('progress:', epoch, l.item()) # 取出loss使用l.item，不要直接使用l（l是tensor会构建计算图）
 
print("predict (after training)", 4, forward(4).item())


predict (before training) 4 4.0
torch.FloatTensor
torch.FloatTensor
progress: 0 1.0
torch.FloatTensor
torch.FloatTensor
progress: 1 1.0
torch.FloatTensor
torch.FloatTensor
progress: 2 1.0
torch.FloatTensor
torch.FloatTensor
progress: 3 1.0
torch.FloatTensor
torch.FloatTensor
progress: 4 1.0
torch.FloatTensor
torch.FloatTensor
progress: 5 1.0
torch.FloatTensor
torch.FloatTensor
progress: 6 1.0
torch.FloatTensor
torch.FloatTensor
progress: 7 1.0
torch.FloatTensor
torch.FloatTensor
progress: 8 1.0
torch.FloatTensor
torch.FloatTensor
progress: 9 1.0
predict (after training) 4 4.0


In [ ]:
evidence = np.array([19]*12)
print(evidence)

[19 19 19 19 19 19 19 19 19 19 19 19]


In [ ]:
### Loss function
def loss_cal(para):
    y_label = np.array([1,0,0,0,0,0,0,0,0,0,0,0])
    
    # parameters: alphas
    S = np.sum(para)
    pred = para/S
    pred_err = np.sum(np.square(pred-y_label))

    pred_var = np.sum(pred*(1-pred)/(S+1))
    #pred_var = np.sum(pred*(1-pred))
    return pred_err,pred_var


In [ ]:
def kl_divergence(para,n_class=3):
    beta = np.ones()


In [ ]:
alphas = np.array([10,12,10,10,10,10,10,10,10,10,10,10])
pred_err,pred_var = loss_cal(alphas)
print(pred_err,pred_var,pred_err+pred_var)

0.9196452566514377 0.007450571683894748 0.9270958283353324


In [ ]:
prob = np.array([0,0.5,0])
print(mse_loss(prob))
prob = np.array([0,0.4,0.3])
print(mse_loss(prob))

1.25
1.2500000000000002


In [ ]:
alphas = np.array([1,10,8])
alphas = (alphas-1)*(1-np.array([1,0,0]))+1
print(alphas)
beta = np.ones(3)
#beta = np.array([8,8,8])
S_alpha = np.sum(alphas)
print(S_alpha)
S_beta = np.sum(beta)
print(S_beta)
lnB = sc.gammaln(S_alpha) - np.sum(sc.gammaln(alphas))
print(lnB)
lnB_uni = np.sum(sc.gammaln(beta)) - sc.gammaln(S_beta)
print(lnB_uni)
dg0 = sc.digamma(S_alpha)
dg1 = sc.digamma(alphas)
kl = np.sum((alphas-beta)*(dg1-dg0))+lnB+lnB_uni
print(kl)

[ 1 10  8]
19
3.0
15.068456366886167
-0.6931471805599453
2.0642942208994857


In [ ]:
y = np.array([1,0,0])
kl_alpha = (alphas - 1) * (1 - y) + 1
print(kl_alpha)

[1 3 1]


In [ ]:
import torch
#alpha = torch.ones([1, 12], dtype=torch.float32)
alpha = torch.tensor([[1,20,1,1,18,1,1,18,1,1,1,1]], dtype=torch.float32)
beta = torch.ones([1, 12], dtype=torch.float32)
S_alpha = torch.sum(alpha, dim=1, keepdim=True)
print(S_alpha)
S_beta = torch.sum(beta, dim=1, keepdim=True)
print(S_beta)
lnB = torch.lgamma(S_alpha) - \
    torch.sum(torch.lgamma(alpha), dim=1, keepdim=True)
print(lnB)
lnB_uni = torch.sum(torch.lgamma(beta), dim=1,
                    keepdim=True) - torch.lgamma(S_beta)
print(lnB_uni)
dg0 = torch.digamma(S_alpha)
dg1 = torch.digamma(alpha)

kl = torch.sum((alpha - beta) * (dg1 - dg0), dim=1,
                keepdim=True) + lnB + lnB_uni

print(kl)

tensor([[65.]])
tensor([[12.]])
tensor([[98.8182]])
tensor([[-17.5023]])
tensor([[14.2415]])


In [ ]:
y = torch.tensor([[1,0,0,0,0,0,0,0,0,0,0,0]], dtype=torch.float32)
#print(1-torch.sum(alpha*y,dim=1,keepdim=True)/torch.sum(alpha, dim=1, keepdim=True))

In [ ]:
y2 = torch.tensor([[2,3,0,0,0,0,0,0,0,0,0,0]], dtype=torch.float32)
#print(torch.dot(y2,y))

RuntimeError: ignored

In [ ]:
y3 = torch.tensor([[2],[3],[4]])
print(y2.reshape(3,4)/y3)

tensor([[1.0000, 1.5000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])


In [ ]:
value = 5
temp = y[y!=torch.max(y)]
print(temp)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [ ]:
a = [1,2,3]
a[0],a[1],a[2] = a[2],a[1],a[0]
print(a)


[3, 2, 1]


In [ ]:
import torch
import matplotlib.pyplot as plt  
a = torch.linspace(0, 1, 10, requires_grad=True)
b = torch.tensor([0.5], requires_grad=True)
loss = torch.abs(a-b)/(a+b+1e-8)
loss.backward() 
plt.plot(a.detach().numpy(), a.grad.detach().numpy(), label='grad')
plt.legend()

tensor([0.0000, 0.1111, 0.2222, 0.3333, 0.4444, 0.5556, 0.6667, 0.7778, 0.8889,
        1.0000], requires_grad=True)
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  import sys


RuntimeError: ignored

In [ ]:
import torch
a = torch.tensor([0.3],requires_grad=True)
b = torch.tensor([0.2],requires_grad=True)
#loss = torch.where(a*b>0,torch.abs(a-b)/(a+b+1e-8), torch.abs(a-b)/(a+b+1e-8))
#loss = torch.where(a*b>0,torch.abs(a-b)/(a+b+1e-8), torch.zeros(1,1))
loss = 1-torch.abs(a-b)/(a+b+1e-8)
#loss = torch.where(torch.abs(a-b)<=1e-3, 0.5*(a-b)**2/(a+b+1e-8), (1e-3*torch.abs(a-b)-0.5*1e-3**2)/(a+b+1e-8))
loss.backward()
print(a.grad.data)

tensor([-1.6000])


In [ ]:
a = torch.tensor([[10.,2.0,3.0]],requires_grad=True)
b = torch.sum(a,dim=1, keepdim=True)
loss = torch.log(b)
loss.backward()
print(a.grad.data)

tensor([[0.0667, 0.0667, 0.0667]])


In [ ]:
a = torch.tensor([0.4],requires_grad=True)
loss = (a-0.5)/(a+0.5)
loss.backward()
print(a.grad.data)

tensor([1.2346])


In [ ]:
index = y>0
y2 = torch.tensor([[2,0.5,0,0,0,0,0,0,0,0,0,0]])
print(index*y+y2*(~index))

tensor([[1.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000]])


In [ ]:
outputs = torch.tensor([[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0,0],[1,0.9,0.8,0.7,0.6,5.5,0.4,0.3,0.2,0.1,0,0]], dtype=torch.float32)
print(outputs.size())
_, preds = torch.max(outputs, 1)
print(preds)

print(torch.eq(preds, preds))

mean = torch
var =  torch.tensor([[0.1,0.3]], dtype=torch.float32)
index = var > 0.5
print(index)
index = mean + var > 0.5
print(index)
print(mean + var)
_, preds = torch.max(outputs, 1)

torch.Size([2, 12])
tensor([0, 5])
tensor([True, True])


'\nvar =  torch.tensor([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1,0.1,0.1,0.1]], dtype=torch.float32)\nindex = var > 0.5\nprint(index)\nindex = mean + var > 0.5\nprint(index)\nprint(mean + var)\n\n_, preds = torch.max(outputs, 1)\n'

In [ ]:
y = torch.tensor([[1,0,0],[1,0,0],[1,0,0]], dtype=torch.float32)

In [ ]:
def get_device():
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    return device

def loglikelihood_loss(y, alpha, device=None):
    if not device:
        device = get_device()
    y = y.to(device)
    alpha = alpha.to(device)
    S = torch.sum(alpha, dim=1, keepdim=True)
    loglikelihood_err = torch.sum(
        (y - (alpha / S)) ** 2, dim=1, keepdim=True)
    loglikelihood_var = torch.sum(
        alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)
    #loglikelihood = loglikelihood_err + loglikelihood_var
    return loglikelihood_err,loglikelihood_var

def kl_divergence(alpha, num_classes, device=None):
    if not device:
        device = get_device()
    beta = torch.ones([1, num_classes], dtype=torch.float32, device=device)
    S_alpha = torch.sum(alpha, dim=1, keepdim=True)
    S_beta = torch.sum(beta, dim=1, keepdim=True)
    lnB = torch.lgamma(S_alpha) - \
        torch.sum(torch.lgamma(alpha), dim=1, keepdim=True)
    lnB_uni = torch.sum(torch.lgamma(beta), dim=1,
                        keepdim=True) - torch.lgamma(S_beta)

    dg0 = torch.digamma(S_alpha)
    dg1 = torch.digamma(alpha)

    kl = torch.sum((alpha - beta) * (dg1 - dg0), dim=1,
                   keepdim=True) + lnB + lnB_uni
    return kl

In [ ]:
import torch.nn.functional as F
y = torch.tensor([[1,0,-1],[0,1,0],[0,0,1]]*3, dtype=torch.float32)
print(F.softmax(y,dim=1))


tensor([[0.6652, 0.2447, 0.0900],
        [0.2119, 0.5761, 0.2119],
        [0.2119, 0.2119, 0.5761],
        [0.6652, 0.2447, 0.0900],
        [0.2119, 0.5761, 0.2119],
        [0.2119, 0.2119, 0.5761],
        [0.6652, 0.2447, 0.0900],
        [0.2119, 0.5761, 0.2119],
        [0.2119, 0.2119, 0.5761]])


In [ ]:
y2 = torch.tensor([1,2,3,4,5,6,7,8,9]).reshape(9,1)
print(y*y2)


tensor([[1., 0., 0.],
        [0., 2., 0.],
        [0., 0., 3.],
        [4., 0., 0.],
        [0., 5., 0.],
        [0., 0., 6.],
        [7., 0., 0.],
        [0., 8., 0.],
        [0., 0., 9.]])


In [ ]:
temp = [[3,1,1], # ideal
        [1,2,1], # ideal
        [1,1,3], # ideal
        [10,8,1], # practial
        [8,10,1], # practical 
        [1,2,3], # practical
        [1,4,1], # wrong 
        [3,1,1], # wrong 
        [1,2,1]  # wrong 
        ]


In [ ]:
alpha = torch.tensor(temp, dtype=torch.float32)
#A_err,A_var = loglikelihood_loss(y, alpha)
kl_alpha = (alpha - 1) * (1 - y) + 1
kl_div = kl_divergence(kl_alpha, 3)
#loss = torch.mean(A+kl_div)


RuntimeError: ignored

In [ ]:
print(A_err)
print(kl_div)
print(kl_div+A_err)
print(torch.mean(A_err+kl_div))

tensor([[0.2400],
        [0.3750],
        [0.2400],
        [0.3889],
        [0.2857],
        [0.3889],
        [1.1667],
        [1.0400],
        [0.8750]])
tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.2653],
        [0.2653],
        [0.2653],
        [0.9526],
        [0.6251],
        [0.2653]])
tensor([[0.2400],
        [0.3750],
        [0.2400],
        [0.6542],
        [0.5510],
        [0.6542],
        [2.1193],
        [1.6651],
        [1.1403]])
tensor(0.8488)


In [ ]:
y = torch.tensor([[1.0,2.0],[2.0,0.5],[0.2,3]], dtype=torch.float32)
label = torch.tensor([1,0,0])
_,p = torch.max(y,1)
print(torch.where(label==p,1,-1))

tensor([ 1,  1, -1])


In [ ]:
y[:,-1].reshape(3,1).size()
#print(a)

torch.Size([3, 1])

In [ ]:
y[:,:-1]

tensor([[1.0000],
        [2.0000],
        [0.2000]])